In [1]:
#Imports
import numpy as np
import matplotlib.pyplot as plt

In [11]:
SFR  = 1.7

Mag = 21
MagFilt = 'r'

texp = 30000 # seconds

Dist = 14.5e6 * 3.086e18 # Distance in cm
Diam = 4 # Diameter in arcsec

RON = 1.5 #e-
DARKC = 0.002 # e-/s/pixel
GAIN=0.25 #e-/ADU at G = 125
PSCALE = 0.55 # arcsec/pixel

zp = {'Ha':-14.73, 'Hb':-14.85, 'OIII':-14.77, 'SII':-14.78, 'g':-16.26, 'r':-16.26, 'i':-16.06} # cgs zeropoint for all filters
skybright = {'Ha': 5.47e-16, 'Hb':8.11e-16, 'OIII':8.11e-16, 'SII':5.47e-16, 'g':8.11e-16, 'r':5.47e-16, 'i':2.15e-16} # erg/s/cm^2/arcsec^2

SNRarea = (4)**2 # Area of the SNR region
SNRarea_pix = SNRarea / (PSCALE**2) #find number of pixels in SNR area
print(SNRarea_pix)

52.89256198347107


In [3]:
#SN equation for t a given time
def SNRcalc(t, flux, skyflux, npix, zp): #t in seconds, flux in erg/s/cm^2/pixel, npix in pixels, zp in flambda cgs

    Nstar = npix * t * flux / 10**zp
    Nsky = npix * t * skyflux / 10**zp
    Ndark = npix * DARKC * t
    Nron = npix * RON**2

    #print(Nstar, Nsky, Ndark, Nron)

    Den = np.sqrt(Nstar+ Nsky + Ndark + Nron)

    SNR = Nstar/Den

    return SNR

In [37]:
if SFR is not None:
    L_Ha_per_SFR = 1.26e41 # erg/s per (solar mass per year)
    L_Ha = SFR * L_Ha_per_SFR # erg/s

    F_Ha = L_Ha / (4 * np.pi * Dist**2) / 35 # erg/s/cm^2/A
    Area = np.pi * (Diam / 2)**2 # arcsec^2
    SB_Ha = F_Ha / Area # erg/s/cm^2/arcsec^2

    print(f"Expected H-alpha flux density from NGC 6946: {F_Ha:.2e} erg/s/cm^2/A")
    print(f"Expected H-alpha surface brightness from NGC 6946: {SB_Ha:.2e} erg/s/cm^2/A/arcsec^2")

    SNRHa = SNRcalc(texp,SB_Ha*PSCALE**2,skybright['Ha']*PSCALE**2,SNRarea_pix,zp['Ha'])
    SNRHb = SNRcalc(texp,(SB_Ha/3)*PSCALE**2,skybright['Hb']*PSCALE**2,SNRarea_pix,zp['Hb'])

    print('SNR Ha in {} s: {}'.format(texp, SNRHa))
    print('SNR Hb in {} s: {}'.format(texp, SNRHb))

else:
    print('Populate the SFR variable to calculate expected fluxes and SNRs')

Expected H-alpha flux density from NGC 6946: 2.43e-13 erg/s/cm^2/A
Expected H-alpha surface brightness from NGC 6946: 3.13e-19 erg/s/cm^2/A/arcsec^2
SNR Ha in 1200 s: 10.45466692678259
SNR Hb in 1200 s: 3.320785276304476


In [12]:
if Mag is not None and MagFilt is not None:

    fnu = 3631 * 10**(-0.4*Mag) * 1e-23 # erg/s/cm^2/Hz
    c = 3e18 # A/s
    if MagFilt == 'g':
        lambda_eff = 4770 # cm
    elif MagFilt == 'r':
        lambda_eff = 6230 # cm
    elif MagFilt == 'i':
        lambda_eff = 7625 # cm
    flambda = fnu * c / lambda_eff**2 # erg/s/cm^2

    print(flambda)

    Area = np.pi * (Diam / 2)**2 # arcsec^2
    SB_filt = flambda / Area # erg/s/cm^2/A/arcsec^2

    SNRHa = SNRcalc(texp,SB_filt*PSCALE**2,skybright[MagFilt]*PSCALE**2,SNRarea_pix,zp[MagFilt])

    print('SNR in filter {} in {} s: {}'.format(MagFilt, texp, SNRHa))

else:
    print('Populate the Mag and Magfilt variable to calculate expected fluxes and SNRs')

1.1173041460028086e-17
SNR in filter r in 30000 s: 3.548840460975218
